ToDos:

DONE: Add bias unit to backpropogation/updates (as w[0], or as a distinct variable?)

DONE: Allow for variable cost functions

DONE: Add optional internal regularization of inputs? (Or is this better kept external?)

    (Added external function for regularizing inputs befoer plugging them into network)
- Add check for determining when training is complete?
- Why can't it work with a single training example? Where is error coming from? I still don't get it :(

In [13]:
import numpy as np

### NETWORK CLASSES

class nNetwork(object):
    def __init__(self, numX, numY, hlayers, activation, cost=(lambda Y, o : Y - o)):
        # Define activation function. 
        # Expects 'activation' to be a class with forward() and backward() static methods
        self.activation = activation
        
        # Expects a function that can take labels, outputs, and return error
        self.cost = cost
        
        # Initialize network layers as a list of nlayer objects
        # Expects 'hlayers' to be a list of integers; each being the size of each layer
        self.layer = [ nLayer(i, activation) for i in hlayers ]
        self.layer.insert(0, nLayer(numX, activation))
        self.layer.append(nLayer(numY, activation))
        
        # Initialize WEIGHT MATRICIES and BIASES for each layer
        # WEIGHT MATRICIES are the theta multipliers applied to the
        # PREVIOUS layer's output to determine the CURRENT layer's activations
        # Each node is represented by a row; each input is a column
        for i in range(1, len(self.layer)):
            prevnodes = self.layer[i-1].len
            curnodes = self.layer[i].len
            # Add an additional set of weights to serve as biases for each node in this layer
            self.layer[i].w = np.random.randn(prevnodes + 1, curnodes)
    
    def forward(self, X):
        # Input layer activation
        a = self.layer[0].input_activation(X)
        
        # Hidden and output layer activation
        for l in self.layer[1:]:
            a = l.activation(a)
        
        # Clean bias unit from output layer
        self.layer[-1].a = np.delete(self.layer[-1].a, 0, axis=1) 
        return a
    
    def backward(self, Y, rate):
        prime = self.activation.backward
        cost = self.cost
        
        # Calculate delta for all layers
        for l in self.layer[::-1]:
            if l == self.layer[-1]:
                # For output layer:
                # Calculate error (labels - predictions)
                # Calculate delta
                l.error = cost(Y, l.a)
                l.delta = l.error*prime(l.a)
                pre_d = l.delta
                pre_w = l.w
                
            elif l == self.layer[0]:
                # For input layer:
                # Do nothing
                pass
            
            else:
                # For hidden layers:
                # Calculate error (apply succeeding layer's delta to adjoined weights)
                # Calculate delta by applying error to derivative of layer's output
                l.error = pre_d @ pre_w.T[:, 1:]    # Don't backpropogate from the Bias node!
                l.delta = l.error*prime(l.a[:, 1:]) # Don't backpropogate from the Bias node!
                pre_d = l.delta
                pre_w = l.w
        
        # Apply delta for all layers
        for l in self.layer:
            if l == self.layer[0]:
                # For input layer:
                # Do nothing
                pre_a = l.a
            else:
                # For hidden & output layers:
                # Set weights to weights + deltas * inputs * learning rate
                l.w += (pre_a.T @ l.delta) * rate
                pre_a = l.a          
            
            
    def train (self, X, Y, epochs, rate=1):
        print(f'Training for {epochs} epochs at α={rate}:')
        for i in range(0,epochs):
            a = self.forward(X)
            self.backward(Y, rate)
            print(f'Epoch {i+1} SSq Loss: {np.mean(np.square(Y - a))}')
    
    def describe(self):
        # Descriptive tool for displaying network's size, shape, etc.
        out = ''
        
        print(f'Input Nodes (X): {self.layer[0].len}')
        print(f'Output Nodes (Y): {self.layer[-1].len}')
        print(f'Hidden Layers: {len(self.layer[1:-1])} Total')
        print(f'   Sizes: {[l.len for l in self.layer[1:-1]]}')
        print('\nNetwork Shape:')
        for index, item in enumerate(self.layer):
            if index == 0:
                out += str(item.len) + 'X '
            elif index == len(self.layer)-1:
                out += '| ' + str(item.len) + 'Y'
            else:
                out += '| ' + str(item.len) + ' '
        print(out)
        

class nLayer(object):
    def __init__(self, nodecount, activation):
        self.len = nodecount         # Number of nodes/perceptrons. Used for reports. 
        self.w = None                # Weight matrix (declared in nNetwork object)
        self.z = None                # (Dot product of inputs x weight matrix) + bias'
        self.a = None                # 1D array of unit outputs
        self.error = None            # Layer error
        self.delta = None            # 1D array of deltas, for backprop
        self.g = activation.forward  # Layer activation function
        
    def activation(self, i):
        self.z = i @ self.w
        self.a = self.g(self.z)
        self.a = np.hstack((np.ones((np.size(self.a,0),1)), self.a))
        return self.a
    
    def input_activation(self, i):
        self.z = i
        self.a = self.z
        self.a = np.hstack((np.ones((np.size(self.a,0),1)), self.a))
        return self.a
    
    def describe(self):
        print(f'Layer Nodes: {self.len}')
        print(f'\nw (input weights): {len(self.w)} (inputs) x {len(self.w.T)} (nodes)\n{self.w}')
        print(f'\nz (weighted sums): {len(self.z)}\n{self.z}')
        print(f'\na (outputs): {len(self.a)}\n{self.a}')
        print(f'\nerrors: {len(self.error)}\n{self.error}')
        print(f'\ndeltas: {len(self.delta)}\n{self.delta}')
        print(f'\nactivation function: {self.g}')

        
        
### ACTIVATION FUNCTIONS (CLASSES)       

class sigmoid(object):
    @staticmethod
    def forward(x):
        return 1/(1+np.exp(-x))
    
    # TECHINICALLY, sigmoid' is sigmoid(x) * (1 - sigmoid(x))
    # However, we are applying this to the nNetwork.layer.a values, 
    # which have already had the sigmoid function applied to them.
    # Therefore, we will use x * (1 - x) for backpropogation.
    @staticmethod
    def backward(x):
        return x * (1 - x)

    
    
    
### COST FUNCTIONS

def crossentropy(Y, o):
    # Cost funtion for use in classification problems
    if Y == 1:
        return -(np.log(o))
    else:
        return -(np.log(1 - o))
    
    
    
### SUPPORT FUNCTIONS

def scale(X, Y, Xmax=None, Ymax=None):
    if not Xmax:
        Xmax = np.amax(X, axis=0)
    if not Ymax:
        Ymax = np.amax(Y, axis=0)
    
    X_b = X/Xmax
    Y_b = Y/Ymax
    
    return X_b, Y_b

In [14]:
X1 = np.array(([2, 9], [1, 5]), dtype=float)
y1 = np.array(([92], [86]), dtype=float)

# scale units
X1, y1 = scale(X1, y1, Ymax=100)

test1 = nNetwork(2,1,[4,6,4],sigmoid)
test1.train(X1, y1, 1000)

Training for 1000 epochs at α=1:
Epoch 1 Loss: 0.20075869227062476
Epoch 2 Loss: 0.1410189076591983
Epoch 3 Loss: 0.08489711804201441
Epoch 4 Loss: 0.047677578493861034
Epoch 5 Loss: 0.028898445123512917
Epoch 6 Loss: 0.01995162910869915
Epoch 7 Loss: 0.015357925867988464
Epoch 8 Loss: 0.01276446251977115
Epoch 9 Loss: 0.011178417581281238
Epoch 10 Loss: 0.01014603984427185
Epoch 11 Loss: 0.009440710213855261
Epoch 12 Loss: 0.00894010645884216
Epoch 13 Loss: 0.008573798269424455
Epoch 14 Loss: 0.008299013723254325
Epoch 15 Loss: 0.008088602946803136
Epoch 16 Loss: 0.007924681513685724
Epoch 17 Loss: 0.0077950927769651
Epoch 18 Loss: 0.007691348610309258
Epoch 19 Loss: 0.007607383398331416
Epoch 20 Loss: 0.007538774314962442
Epoch 21 Loss: 0.007482238818556664
Epoch 22 Loss: 0.007435302294772414
Epoch 23 Loss: 0.007396073119859858
Epoch 24 Loss: 0.007363087268079259
Epoch 25 Loss: 0.007335198966827321
Epoch 26 Loss: 0.00731150246566696
Epoch 27 Loss: 0.007291275217502467
Epoch 28 Loss: 

Epoch 429 Loss: 0.007046838725496881
Epoch 430 Loss: 0.007046649555013772
Epoch 431 Loss: 0.0070464607393140475
Epoch 432 Loss: 0.007046272277623594
Epoch 433 Loss: 0.007046084169169589
Epoch 434 Loss: 0.007045896413180502
Epoch 435 Loss: 0.007045709008886106
Epoch 436 Loss: 0.007045521955517447
Epoch 437 Loss: 0.007045335252306887
Epoch 438 Loss: 0.007045148898488077
Epoch 439 Loss: 0.00704496289329597
Epoch 440 Loss: 0.007044777235966807
Epoch 441 Loss: 0.007044591925738142
Epoch 442 Loss: 0.0070444069618488155
Epoch 443 Loss: 0.007044222343538989
Epoch 444 Loss: 0.0070440380700501045
Epoch 445 Loss: 0.007043854140624919
Epoch 446 Loss: 0.007043670554507494
Epoch 447 Loss: 0.0070434873109431854
Epoch 448 Loss: 0.007043304409178668
Epoch 449 Loss: 0.007043121848461911
Epoch 450 Loss: 0.0070429396280421944
Epoch 451 Loss: 0.007042757747170112
Epoch 452 Loss: 0.007042576205097547
Epoch 453 Loss: 0.0070423950010777125
Epoch 454 Loss: 0.007042214134365117
Epoch 455 Loss: 0.007042033604215

Epoch 877 Loss: 0.006988184397360406
Epoch 878 Loss: 0.006988095083517903
Epoch 879 Loss: 0.006988005896466715
Epoch 880 Loss: 0.006987916835912108
Epoch 881 Loss: 0.006987827901560024
Epoch 882 Loss: 0.0069877390931171015
Epoch 883 Loss: 0.006987650410290655
Epoch 884 Loss: 0.006987561852788676
Epoch 885 Loss: 0.006987473420319854
Epoch 886 Loss: 0.006987385112593536
Epoch 887 Loss: 0.006987296929319756
Epoch 888 Loss: 0.006987208870209229
Epoch 889 Loss: 0.0069871209349733265
Epoch 890 Loss: 0.006987033123324114
Epoch 891 Loss: 0.006986945434974307
Epoch 892 Loss: 0.006986857869637305
Epoch 893 Loss: 0.006986770427027165
Epoch 894 Loss: 0.0069866831068586144
Epoch 895 Loss: 0.006986595908847044
Epoch 896 Loss: 0.006986508832708502
Epoch 897 Loss: 0.006986421878159709
Epoch 898 Loss: 0.006986335044918028
Epoch 899 Loss: 0.006986248332701495
Epoch 900 Loss: 0.0069861617412287965
Epoch 901 Loss: 0.006986075270219268
Epoch 902 Loss: 0.006985988919392911
Epoch 903 Loss: 0.0069859026884703

In [20]:
X1 = np.array(([2, 9], [1, 5]), dtype=float)
y1 = np.array(([92], [86]), dtype=float)

# scale units
X1, y1 = scale(X1, y1, Ymax=100)

test1 = nNetwork(2,1,[50,50,50,50],sigmoid)
test1.train(X1, y1, 5000)

Training for 5000 epochs at α=1:
Epoch 1 Loss: 0.3537977159333583
Epoch 2 Loss: 0.028508107937320115
Epoch 3 Loss: 0.011728260235515537
Epoch 4 Loss: 0.011611764916348731
Epoch 5 Loss: 0.011474831608291644
Epoch 6 Loss: 0.011312448396186068
Epoch 7 Loss: 0.011118196766982258
Epoch 8 Loss: 0.010883966717676628
Epoch 9 Loss: 0.01059985577813707
Epoch 10 Loss: 0.010254703665870142
Epoch 11 Loss: 0.009838309451505475
Epoch 12 Loss: 0.009347385705947626
Epoch 13 Loss: 0.00879796734839284
Epoch 14 Loss: 0.008243116330528655
Epoch 15 Loss: 0.00777717480823825
Epoch 16 Loss: 0.007486813192321976
Epoch 17 Loss: 0.007366900987670061
Epoch 18 Loss: 0.007333821610631386
Epoch 19 Loss: 0.00732434651546933
Epoch 20 Loss: 0.007318753678203693
Epoch 21 Loss: 0.007313659764724848
Epoch 22 Loss: 0.0073086502374166955
Epoch 23 Loss: 0.007303680852887636
Epoch 24 Loss: 0.0072987467562589045
Epoch 25 Loss: 0.007293847023154943
Epoch 26 Loss: 0.007288981101931362
Epoch 27 Loss: 0.007284148472383183
Epoch 28

Epoch 235 Loss: 0.006719770448718567
Epoch 236 Loss: 0.006718764803307794
Epoch 237 Loss: 0.006717766399967791
Epoch 238 Loss: 0.006716775116164637
Epoch 239 Loss: 0.006715790831730321
Epoch 240 Loss: 0.006714813428834138
Epoch 241 Loss: 0.006713842791953433
Epoch 242 Loss: 0.0067128788078439135
Epoch 243 Loss: 0.006711921365509419
Epoch 244 Loss: 0.006710970356171377
Epoch 245 Loss: 0.006710025673237864
Epoch 246 Loss: 0.006709087212272412
Epoch 247 Loss: 0.006708154870962567
Epoch 248 Loss: 0.0067072285490882445
Epoch 249 Loss: 0.006706308148489992
Epoch 250 Loss: 0.006705393573037097
Epoch 251 Loss: 0.00670448472859564
Epoch 252 Loss: 0.006703581522996578
Epoch 253 Loss: 0.006702683866003739
Epoch 254 Loss: 0.006701791669281977
Epoch 255 Loss: 0.006700904846365294
Epoch 256 Loss: 0.0067000233126251335
Epoch 257 Loss: 0.006699146985238765
Epoch 258 Loss: 0.00669827578315785
Epoch 259 Loss: 0.006697409627077162
Epoch 260 Loss: 0.00669654843940351
Epoch 261 Loss: 0.006695692144224877
E

Epoch 485 Loss: 0.00657001546507134
Epoch 486 Loss: 0.006569656618707442
Epoch 487 Loss: 0.006569299230347168
Epoch 488 Loss: 0.0065689432967676945
Epoch 489 Loss: 0.006568588814739789
Epoch 490 Loss: 0.006568235781027751
Epoch 491 Loss: 0.006567884192389344
Epoch 492 Loss: 0.0065675340455757375
Epoch 493 Loss: 0.006567185337331465
Epoch 494 Loss: 0.006566838064394367
Epoch 495 Loss: 0.006566492223495557
Epoch 496 Loss: 0.006566147811359385
Epoch 497 Loss: 0.006565804824703389
Epoch 498 Loss: 0.0065654632602382855
Epoch 499 Loss: 0.0065651231146679435
Epoch 500 Loss: 0.006564784384689353
Epoch 501 Loss: 0.006564447066992604
Epoch 502 Loss: 0.0065641111582609
Epoch 503 Loss: 0.006563776655170527
Epoch 504 Loss: 0.006563443554390851
Epoch 505 Loss: 0.00656311185258433
Epoch 506 Loss: 0.0065627815464065
Epoch 507 Loss: 0.00656245263250599
Epoch 508 Loss: 0.006562125107524528
Epoch 509 Loss: 0.006561798968096961
Epoch 510 Loss: 0.006561474210851255
Epoch 511 Loss: 0.006561150832408525
Epoc

Epoch 742 Loss: 0.006515953276666384
Epoch 743 Loss: 0.0065158535360435995
Epoch 744 Loss: 0.006515754369204023
Epoch 745 Loss: 0.0065156557733284754
Epoch 746 Loss: 0.006515557745607061
Epoch 747 Loss: 0.006515460283239167
Epoch 748 Loss: 0.006515363383433486
Epoch 749 Loss: 0.006515267043408023
Epoch 750 Loss: 0.0065151712603901115
Epoch 751 Loss: 0.006515076031616426
Epoch 752 Loss: 0.0065149813543329835
Epoch 753 Loss: 0.0065148872257951695
Epoch 754 Loss: 0.0065147936432677295
Epoch 755 Loss: 0.006514700604024801
Epoch 756 Loss: 0.006514608105349898
Epoch 757 Loss: 0.006514516144535941
Epoch 758 Loss: 0.006514424718885252
Epoch 759 Loss: 0.006514333825709563
Epoch 760 Loss: 0.006514243462330033
Epoch 761 Loss: 0.006514153626077242
Epoch 762 Loss: 0.0065140643142912085
Epoch 763 Loss: 0.006513975524321381
Epoch 764 Loss: 0.006513887253526662
Epoch 765 Loss: 0.006513799499275395
Epoch 766 Loss: 0.006513712258945381
Epoch 767 Loss: 0.0065136255299238785
Epoch 768 Loss: 0.006513539309

Epoch 1003 Loss: 0.006502855590477142
Epoch 1004 Loss: 0.006502836139345119
Epoch 1005 Loss: 0.0065028168168945345
Epoch 1006 Loss: 0.006502797622313848
Epoch 1007 Loss: 0.006502778554796229
Epoch 1008 Loss: 0.0065027596135395225
Epoch 1009 Loss: 0.006502740797746234
Epoch 1010 Loss: 0.006502722106623507
Epoch 1011 Loss: 0.006502703539383094
Epoch 1012 Loss: 0.006502685095241339
Epoch 1013 Loss: 0.006502666773419155
Epoch 1014 Loss: 0.006502648573141996
Epoch 1015 Loss: 0.006502630493639844
Epoch 1016 Loss: 0.006502612534147176
Epoch 1017 Loss: 0.0065025946939029486
Epoch 1018 Loss: 0.006502576972150575
Epoch 1019 Loss: 0.0065025593681379
Epoch 1020 Loss: 0.00650254188111718
Epoch 1021 Loss: 0.0065025245103450605
Epoch 1022 Loss: 0.006502507255082556
Epoch 1023 Loss: 0.0065024901145950215
Epoch 1024 Loss: 0.006502473088152138
Epoch 1025 Loss: 0.006502456175027889
Epoch 1026 Loss: 0.006502439374500537
Epoch 1027 Loss: 0.006502422685852601
Epoch 1028 Loss: 0.006502406108370839
Epoch 1029

...

Epoch 3773 Loss: 0.006500000000006604
Epoch 3774 Loss: 0.006500000000006557
Epoch 3775 Loss: 0.00650000000000651
Epoch 3776 Loss: 0.006500000000006463
Epoch 3777 Loss: 0.0065000000000064164
Epoch 3778 Loss: 0.0065000000000063705
Epoch 3779 Loss: 0.0065000000000063245
Epoch 3780 Loss: 0.0065000000000062785
Epoch 3781 Loss: 0.006500000000006234
Epoch 3782 Loss: 0.006500000000006189
Epoch 3783 Loss: 0.006500000000006145
Epoch 3784 Loss: 0.0065000000000061
Epoch 3785 Loss: 0.0065000000000060565
Epoch 3786 Loss: 0.006500000000006013
Epoch 3787 Loss: 0.00650000000000597
Epoch 3788 Loss: 0.006500000000005926
Epoch 3789 Loss: 0.006500000000005884
Epoch 3790 Loss: 0.006500000000005841
Epoch 3791 Loss: 0.0065000000000058
Epoch 3792 Loss: 0.006500000000005758
Epoch 3793 Loss: 0.0065000000000057165
Epoch 3794 Loss: 0.006500000000005676
Epoch 3795 Loss: 0.006500000000005635
Epoch 3796 Loss: 0.006500000000005594
Epoch 3797 Loss: 0.006500000000005554
Epoch 3798 Loss: 0.006500000000005514
Epoch 3799 L

Epoch 4034 Loss: 0.006500000000001003
Epoch 4035 Loss: 0.006500000000000996
Epoch 4036 Loss: 0.0065000000000009885
Epoch 4037 Loss: 0.0065000000000009816
Epoch 4038 Loss: 0.006500000000000975
Epoch 4039 Loss: 0.006500000000000968
Epoch 4040 Loss: 0.006500000000000961
Epoch 4041 Loss: 0.006500000000000954
Epoch 4042 Loss: 0.006500000000000947
Epoch 4043 Loss: 0.006500000000000941
Epoch 4044 Loss: 0.006500000000000934
Epoch 4045 Loss: 0.006500000000000927
Epoch 4046 Loss: 0.00650000000000092
Epoch 4047 Loss: 0.006500000000000914
Epoch 4048 Loss: 0.006500000000000907
Epoch 4049 Loss: 0.006500000000000901
Epoch 4050 Loss: 0.006500000000000894
Epoch 4051 Loss: 0.006500000000000887
Epoch 4052 Loss: 0.006500000000000881
Epoch 4053 Loss: 0.006500000000000875
Epoch 4054 Loss: 0.006500000000000868
Epoch 4055 Loss: 0.006500000000000862
Epoch 4056 Loss: 0.006500000000000856
Epoch 4057 Loss: 0.006500000000000849
Epoch 4058 Loss: 0.006500000000000844
Epoch 4059 Loss: 0.006500000000000838
Epoch 4060 

Epoch 4293 Loss: 0.006500000000000154
Epoch 4294 Loss: 0.006500000000000152
Epoch 4295 Loss: 0.0065000000000001515
Epoch 4296 Loss: 0.006500000000000151
Epoch 4297 Loss: 0.006500000000000149
Epoch 4298 Loss: 0.006500000000000149
Epoch 4299 Loss: 0.006500000000000147
Epoch 4300 Loss: 0.006500000000000146
Epoch 4301 Loss: 0.006500000000000145
Epoch 4302 Loss: 0.0065000000000001446
Epoch 4303 Loss: 0.006500000000000143
Epoch 4304 Loss: 0.006500000000000142
Epoch 4305 Loss: 0.006500000000000141
Epoch 4306 Loss: 0.00650000000000014
Epoch 4307 Loss: 0.006500000000000139
Epoch 4308 Loss: 0.006500000000000138
Epoch 4309 Loss: 0.006500000000000138
Epoch 4310 Loss: 0.006500000000000136
Epoch 4311 Loss: 0.006500000000000135
Epoch 4312 Loss: 0.006500000000000134
Epoch 4313 Loss: 0.006500000000000132
Epoch 4314 Loss: 0.006500000000000132
Epoch 4315 Loss: 0.0065000000000001315
Epoch 4316 Loss: 0.006500000000000131
Epoch 4317 Loss: 0.006500000000000129
Epoch 4318 Loss: 0.006500000000000128
Epoch 4319

Epoch 4551 Loss: 0.006500000000000023
Epoch 4552 Loss: 0.006500000000000022
Epoch 4553 Loss: 0.006500000000000022
Epoch 4554 Loss: 0.006500000000000022
Epoch 4555 Loss: 0.006500000000000022
Epoch 4556 Loss: 0.006500000000000022
Epoch 4557 Loss: 0.006500000000000022
Epoch 4558 Loss: 0.006500000000000022
Epoch 4559 Loss: 0.006500000000000022
Epoch 4560 Loss: 0.006500000000000022
Epoch 4561 Loss: 0.006500000000000021
Epoch 4562 Loss: 0.006500000000000021
Epoch 4563 Loss: 0.006500000000000021
Epoch 4564 Loss: 0.006500000000000021
Epoch 4565 Loss: 0.006500000000000021
Epoch 4566 Loss: 0.006500000000000021
Epoch 4567 Loss: 0.00650000000000002
Epoch 4568 Loss: 0.00650000000000002
Epoch 4569 Loss: 0.00650000000000002
Epoch 4570 Loss: 0.00650000000000002
Epoch 4571 Loss: 0.00650000000000002
Epoch 4572 Loss: 0.00650000000000002
Epoch 4573 Loss: 0.00650000000000002
Epoch 4574 Loss: 0.00650000000000002
Epoch 4575 Loss: 0.00650000000000002
Epoch 4576 Loss: 0.006500000000000019
Epoch 4577 Loss: 0.00

Epoch 4817 Loss: 0.006500000000000002
Epoch 4818 Loss: 0.006500000000000002
Epoch 4819 Loss: 0.006500000000000002
Epoch 4820 Loss: 0.006500000000000002
Epoch 4821 Loss: 0.006500000000000002
Epoch 4822 Loss: 0.006500000000000002
Epoch 4823 Loss: 0.006500000000000002
Epoch 4824 Loss: 0.006500000000000002
Epoch 4825 Loss: 0.006500000000000002
Epoch 4826 Loss: 0.006500000000000002
Epoch 4827 Loss: 0.006500000000000002
Epoch 4828 Loss: 0.006500000000000002
Epoch 4829 Loss: 0.006500000000000002
Epoch 4830 Loss: 0.006500000000000002
Epoch 4831 Loss: 0.006500000000000002
Epoch 4832 Loss: 0.006500000000000002
Epoch 4833 Loss: 0.006500000000000002
Epoch 4834 Loss: 0.006500000000000002
Epoch 4835 Loss: 0.006500000000000002
Epoch 4836 Loss: 0.006500000000000002
Epoch 4837 Loss: 0.006500000000000002
Epoch 4838 Loss: 0.006500000000000002
Epoch 4839 Loss: 0.006500000000000002
Epoch 4840 Loss: 0.006500000000000002
Epoch 4841 Loss: 0.006500000000000002
Epoch 4842 Loss: 0.006500000000000002
Epoch 4843 L